In [0]:
balances_df = spark.read.format("delta").load("/Volumes/workspace/recargapay/vol_rp/silver/daily_balance_table")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, lit, when, expr
from pyspark.sql.window import Window

# Start Spark session
spark = SparkSession.builder.appName("InterestCalculation").getOrCreate()

# Parameters (passed externally or set here for testing)
rate_above_100 = 0.001  # 0.1% daily
rate_unmoved = 0.0005   # 0.05% daily

# Example: balances_df must be loaded or created before this
# Columns: account_id, user_id, date, daily_amount, daily_balance

# 1. Window spec for lag
window_spec = Window.partitionBy("user_id").orderBy("date")

# 2. Add previous day’s daily_amount to check inactivity
df_with_lag = balances_df.withColumn(
    "prev_daily_amount", lag("daily_amount").over(window_spec)
)

# 3. Determine interest eligibility and apply respective rates
df_interest = df_with_lag.withColumn(
    "interest_above_100", 
    when(col("daily_balance") > 100, col("daily_balance") * lit(rate_above_100)).otherwise(0.0)
).withColumn(
    "interest_unmoved",
    when((col("daily_amount") == 0) & (col("prev_daily_amount") == 0),
         col("daily_balance") * lit(rate_unmoved)).otherwise(0.0)
).withColumn(
    "total_interest", col("interest_above_100") + col("interest_unmoved")
)

# 4. Select final interest payout
result_df = df_interest.select(
    "account_id", "user_id", "date", "daily_balance",
    "interest_above_100", "interest_unmoved", "total_interest"
)

# Show result
result_df.orderBy("account_id", "date").display()

In [0]:
# Write to Delta format
result_df.write.mode("overwrite").format("delta").save("/Volumes/workspace/recargapay/vol_rp/gold/tb_daily_payment")